In [9]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model

In [10]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [11]:
model = load_model('out/models/model2.h5')

In [12]:
include_points = [23,24,25,26,27,28]

In [13]:
def calulate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180.0:
        angle = 360-angle

    return angle

In [14]:
def calculate_disance(a,b):
    a = np.array(a)
    b = np.array(b)

    return np.linalg.norm(a-b)

In [17]:
# cap = cv2.VideoCapture("http://192.168.0.3:4747/video")
cap = cv2.VideoCapture("vids/incorrect/1.mp4")
reps=0
remark="Incomplete"
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if ret==False:
            break

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)
        left_shoulder = [results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x, results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y]
        right_shoulder = [results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x, results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y]
        shoulder_distance = calculate_disance(left_shoulder, right_shoulder)

        row=[]

        for pt in include_points:
            row.append(results.pose_landmarks.landmark[pt].x/shoulder_distance)
            row.append(results.pose_landmarks.landmark[pt].y/shoulder_distance)

        row=np.array(row)

        probability = model.predict(row.reshape(1,12))
        print(probability.shape)
        probability = probability[0][0]

        prev_remark = remark
        remark = "Complete" if probability > 0.5 else "Incomplete"
        
        if prev_remark=="Incomplete" and remark=="Complete":
            reps+=1

        #Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        resize = cv2.resize(image, (405, 720))
        # Render detections
        mp_drawing.draw_landmarks(resize, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
        
        cv2.putText(resize, "Reps: "+str(reps), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.putText(resize, remark, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        
        cv2.imshow('MediaPipe Pose', resize)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 265ms/step
(1, 1)
1/1 [==============================] - 0s 20ms/step
(1, 1)
1/1 [==============================] - 0s 19ms/step
(1, 1)
1/1 [==============================] - 0s 20ms/step
(1, 1)
1/1 [==============================] - 0s 25ms/step
(1, 1)
1/1 [==============================] - 0s 24ms/step
(1, 1)
1/1 [==============================] - 0s 24ms/step
(1, 1)
1/1 [==============================] - 0s 21ms/step
(1, 1)
1/1 [==============================] - 0s 17ms/step
(1, 1)
1/1 [==============================] - 0s 21ms/step
(1, 1)
1/1 [==============================] - 0s 23ms/step
(1, 1)
1/1 [==============================] - 0s 21ms/step
(1, 1)
1/1 [==============================] - 0s 18ms/step
(1, 1)
1/1 [==============================] - 0s 20ms/step
(1, 1)
1/1 [==============================] - 0s 21ms/step
(1, 1)
1/1 [==============================] - 0s 19ms/step
(1, 1)
1/1 [==============================] - 0s 20ms/step
(1,

In [ ]:
cap = cv2.VideoCapture("http://192.168.0.3:4747/video")
reps=0
remark="Incomplete"
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if ret==False:
            break

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        try:
            # Make detection
            results = pose.process(image)
            left_shoulder = [results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x, results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y]
            right_shoulder = [results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x, results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y]
            shoulder_distance = calculate_disance(left_shoulder, right_shoulder)

            row=[]

            for pt in include_points:
                row.append(results.pose_landmarks.landmark[pt].x/shoulder_distance)
                row.append(results.pose_landmarks.landmark[pt].y/shoulder_distance)

            row=np.array(row).reshape(1,1,12)

            probability = model.predict(row)
            print(probability.shape)
            probability = probability[0][0]

            prev_remark = remark
            remark = "Complete" if probability > 0.5 else "Incomplete"

            if prev_remark=="Incomplete" and remark=="Complete":
                reps+=1

            #Recolor back to BGR
            image.flags.writeable = True
            # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            resize = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            # resize = cv2.resize(image, (405, 720))
            # Render detections
            # mp_drawing.draw_landmarks(resize, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            #                           mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
            #                           mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

            cv2.putText(resize, "Reps: "+str(reps), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            cv2.putText(resize, remark, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        
        except:
            #Recolor back to BGR
            image.flags.writeable = True
            # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            resize = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            # resize = cv2.resize(image, (405, 720))

        cv2.imshow('MediaPipe Pose', resize)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()